# Quick Start

In this tutorial, we assume that Bridgic is already installed on your system. If that’s not the case, see [Installation](../../../installation).

Let's start by building a simple word learning assistant. You provide a word, and the assistant will generate its derived forms and create sentences with them. This example will also show how to use Bridgic in practice.

## Word learning assistant

### 1. Model Initialization

Before getting started, let's set up our environment. In this quick start, we'll use the integration out of the box.
For an in-depth explanation of model integration, see: [LLM Integration](../../model_integration/llm_integration/).

Execute the following commands in the shell:

```shell
export OPENAI_API_KEY="<your_openai_api_key>"
export OPENAI_MODEL_NAME="<the_model_name>"
```

In [ ]:
# Get the environment variables.
import os

# Import the necessary packages.
from bridgic.core.automa import GraphAutoma, worker
from bridgic.core.model.types import Message, Role

# Here we use OpenAILikeLlm because the package `bridgic-llms-openai-like` is installed automatically 
# when you install Bridgic. This makes sure the OpenAI-like model integration works out of the box.
from bridgic.llms.openai_like import OpenAILikeLlm, OpenAILikeConfiguration


# In this tutorial, we use OpenAI as an example. 
# You can freely replace these model settings to use any LLM provider you like.
_api_key = os.environ.get("OPENAI_API_KEY")
_api_base = os.environ.get("OPENAI_API_BASE")
_model_name = os.environ.get("OPENAI_MODEL_NAME")

llm = OpenAILikeLlm(
    api_key=_api_key,
    api_base=_api_base,
    configuration=OpenAILikeConfiguration(model=_model_name),
    timeout=20,
)

### 2. Automa Orchestration

There are two steps to complete the word learning assistant:

1. Generate derivatives of the input word.
2. Make sentences with derivatives.

In [19]:
class WordLearningAssistant(GraphAutoma):
    @worker(is_start=True)
    async def generate_derivatives(self, word: str):
        print(f"------Generating derivatives for {word}------")
        response = await llm.achat(
            model=_model_name,
            messages=[
                Message.from_text(text="You are a word learning assistant. Generate derivatives of the input word in a list.", role=Role.SYSTEM),
                Message.from_text(text=word, role=Role.USER),
            ]
        )
        print(response.message.content)
        print(f"------End of generating derivatives------\n")
        return response.message.content

    @worker(dependencies=["generate_derivatives"], is_output=True)
    async def make_sentences(self, derivatives):
        print(f"------Making sentences with------")
        response = await llm.achat(
            model=_model_name,
            messages=[
                Message.from_text(text="You are a word learning assistant. Make sentences with the input derivatives in a list.", role=Role.SYSTEM),
                Message.from_text(text=derivatives, role=Role.USER),
            ]
        )
        print(response.message.content)
        print(f"------End of making sentences------\n")
        return response.message.content

word_learning_assistant = WordLearningAssistant()

### 3. Agent Running

Let's run this assistant, via [`arun`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma.arun) method:

In [20]:
res = await word_learning_assistant.arun(word="happy")

------Generating derivatives for happy------
Here are some derivatives of the word "happy":

1. Happiness
2. Happily
3. Happier
4. Happiest
5. Unhappy
6. Unhappiness
7. Happinesses (plural form)
8. Happifying (gerund form)
9. Happify (verb form)

Feel free to ask for derivatives of another word!
------End of generating derivatives------

------Making sentences with------
Sure! Here are sentences using each of the derivatives of the word "happy":

1. **Happiness**: The pursuit of happiness is a common goal for many people.
2. **Happily**: She smiled happily as she opened her birthday gifts.
3. **Happier**: After taking a vacation, I felt much happier than I had in months.
4. **Happiest**: That day was the happiest moment of my life when my daughter graduated.
5. **Unhappy**: He seemed unhappy at the party and left early.
6. **Unhappiness**: Her unhappiness was evident in her quiet demeanor.
7. **Happinesses**: Different people find happinesses in various aspects of life, like family, wo

Congratulations! We have successfully completed the word learning assistant, which performed the task exactly according to our requirements.

<div style="text-align: center; margin: 2rem 0;">
<hr style="border: none; border-top: 2px solid #e2e8f0;">
</div>

## What have we done?

The above example idemonstrates a typical way to write an agent application with Bridgic. Let's now explore some of its components.

### Worker

Any callable object (such as functions, methods, etc.) can be converted into a worker object which serve as the **basic execution unit** in Bridgic for scheduling and orchestration. 

Just as in the example of the word learning assistant, we can use a decorator syntax [`@worker`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.worker._worker_decorator.worker) to wrap functions and methods into a worker object.

In [ ]:
class MyAutoma(GraphAutoma):
    @worker(is_start=True)
    async def start(self, x: int):
        return x    

Or, you can also use it like this:

In [ ]:
class MyAutoma(GraphAutoma): ...
my_automa = MyAutoma()

# Add the function as a worker with worker decorator in the instance of the automa
@my_automa.worker(is_start=True)
async def start(x: int):
    return x


Another API [`add_func_as_worker()`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma.add_func_as_worker) can also be used to add workers into a [`GraphAutoma`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma).

In [ ]:
async def start(x: int):
    return x

class MyAutoma(GraphAutoma): ...
my_automa = MyAutoma()

# Add the function as a worker
my_automa.add_func_as_worker(
    key="start",
    func=start,
    is_start=True,
)


In addition to functions being convertible to workers, subclasses that inherit from [`Worker`](../../../../reference/bridgic-core/bridgic/core/automa/worker/#bridgic.core.automa.worker.Worker) and override either [`run()`](../../../../reference/bridgic-core/bridgic/core/automa/worker/#bridgic.core.automa.worker.Worker.run) or [`arun()`](../../../../reference/bridgic-core/bridgic/core/automa/worker/#bridgic.core.automa.worker.Worker.arun) can also be used directly as workers, whose instances can be added into a `GraphAutoma` by the [`add_worker()`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma.add_worker) API.

In [ ]:
from bridgic.core.automa.worker import Worker

class MyWorker(Worker):
    async def arun(self, x: int):
        return x

my_worker = MyWorker()

# Add the worker to the automa
class MyAutoma(GraphAutoma): ...
my_automa = MyAutoma()
my_automa.add_worker(
    key="my_worker",
    worker=my_worker,
    is_start=True,
)

# Run the worker
res = await my_automa.arun(x=1)
print(res)

> Note:
> 1. A specific worker that inherits from `Worker` must override either the `run()` or `arun()` method.
> 2. Bridgic is a framework primarily designed for asynchronous execution, if both `run()` and `arun()` of a worker are overridden, `arun()` will take precedence. Refer to [`Worker`](../../../../reference/bridgic-core/bridgic/core/automa/worker/#bridgic.core.automa.worker.Worker) for details.

In any of these ways the workers can be correctly added into `MyAutoma`. 

Whether using decorator syntax or the corresponding API, there are usually some parameters:

1. `key`: A string used as the worker key. As the unique identifier of a worker in the current automa, it must be ensured that there are no duplicate keys within the same automa. Function or class names are used by default.
2. `func`(in [`add_func_as_worker()`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma.add_func_as_worker)) or `worker`(in [`add_worker()`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma.add_worker)): The actual callable object. **The decorator syntax does not need this parameter.**
3. `is_start`: `True` or `False`. Marking the worker as the start worker of the automa. It can be set for multiple workers.
4. `dependencies`: A list of worker keys. Marking the preceding workers that the worker depends on.
5. `is_output`: `True` or `False`. Marking the worker as the output worker of the automa. Only one output worker can be set per execution branch.
6. `args_mapping_rule`: The [arguments mapping rule](../../../../reference/bridgic-core/bridgic/core/automa/args/#bridgic.core.automa.args.ArgsMappingRule). For detailed information on the parameter binding between workers, please refer to the tutorial: [Parameter Binding](../../core_mechanism/parameter_binding/)

> Note: In Bridgic, a worker must be added to an automa before it can be scheduled and executed. In another word, you shouldn’t directly call `worker.arun()` or `worker.run()` to run a worker.

### GraphAutoma

An automa is an entity that manages and orchestrates a group of workers, serving as the **scheduling engine** . In the example of the word learning assistant above, we used the subclass of [`Automa`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.Automa), i.e. [`GraphAutoma`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma), which performs the scheduling according to the topological sorting among workers.

You should subclass `GraphAutoma` and declare methods as workers with [`@worker`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.worker._worker_decorator.worker).

In [3]:
# Write workers in MyAutoma
class MyAutoma(GraphAutoma):
    @worker(is_start=True)
    async def worker_0(self, a, b, x, y):
        print(f"worker_0: a={a}, b={b}, x={x}, y={y}")

    @worker(is_start=True)
    async def worker_1(self, x, y):
        print(f"worker_1: x={x}, y={y}")

After all the required workers are defined in an automa, the automa can be called with `await automa_obj.arun(*args, **kwargs)` to start the entire scheduling process. 

> Bridgic is a framework built on asynchronous programming. Thus `Graphautoma` must be started using arun(). However, workers may execute in [concurrency mode](../../../../tutorials/items/core_mechanism/concurrency_mode/) when needed.

At startup, the arguments of `automa_obj.arun(*args, **kwargs)` will be distributed to the worker with `is_start=True` according to positional parameters and keyword parameters. 

- positional parameters: The positional arguments passed to `arun` are mapped to the parameters of the workers marked with `is_start=True`, following the order in which they are provided. An error will be raised if the parameter list of some worker is shorter than the number of positional arguments passed to `arun()`.
- keyword parameters: The keyword arguments passed to `arun` are mapped to the corresponding parameters of the workers marked with `is_start=True`.
- priority: **Positional arguments take precedence over keyword arguments.**.

For example: we pass positional arguments `1` and `2`, and keyword arguments `x=3`, `y=4`.

In [4]:
my_automa = MyAutoma()
await my_automa.arun(1, 2, x=3, y=4)

worker_0: a=1, b=2, x=3, y=4
worker_1: x=1, y=2


`1` and `2` were received in order by the first and second parameters of `worker_0` and `worker_1` respectively. Because positional arguments take precedence over keyword arguments, even if the parameter names of `worker_1` are the same as the input keyword parameters, they will still preferentially receive positional arguments.

An error will be raised if the parameter list of some worker is shorter than the number of positional arguments passed to `arun`.

In [ ]:
my_automa = MyAutoma()
await my_automa.arun(1, 2, 3, y=4)  # worker_1 raises an error


If all arguments are passed in keyword format, each worker with `is_start=True` can receive the corresponding values.

In [10]:
my_automa = MyAutoma()
await my_automa.arun(a=1, b=2, x=3, y=4)

worker_0: a=1, b=2, x=3, y=4
worker_1: x=3, y=4


Now, we can start building our Bridgic project!